# Comunicacion de datos

**NOTA: El ambiente de Jupyter Notebook no es el espacio ideal de programación donde codificar programas de comunicación. Esta guía es más un documento de referencia con los códigos a ser probados en un ambiente como Spyder o la consola de comandos.**

Se puede establecer comunicación de datos entre perifericos utilizando diferentes medios y tecnologías. Python tiene diferentes librerias para poder intercambiar datos con otros elementos, utilizando dos tecnologías de transmisión de datos:

* Serial (COM, UART, USB)
* Sockets (Ethernet, WiFi)

## Socket
Un socket es el elemento que utiliza un software para establacer una conexión de red basada en el protocolo IP (ya sea en version 4 o 6) y utilizando protocolos de red (ya sea TCP o UDP). Un socket es una estructura que esta conformada por dos elementos:

* Una direccion de red (IP)
* Un puerto de comunicación (TCP/UDP)

Los métdodos dispoibles en el modulo socket son:

* socket()
* bind()
* listen()
* accept()
* connect()
* connect_ex()
* send()
* recv()
* close()

Un socket se crea utilizando un objeto socket definido con la instrucción socket.socket() y especificando la versión de protocolo IP a utilizar y el protocolo de transmision a usar (TCP: socket.SOCK_STREAM, UDP: SOCK_DGRAM). En los ejemplos utilizaremos el protocolo TCP por ser más estable, confiable y seguro.

En el siguiente diagrama se resume el proceso de llamada a los sockets (socket API) y el flujo de datos TCP en la comunicación entre nodos (en un esquema cliente-servidor, es decir, el nodo "servidor" esta esperando las conexiones de los nodos "cliente" para atender sus requerimientos).

<img src="https://i.imgur.com/zmhwCSA.png" alt="Girl in a jacket" width="700" height="800">

## Echo "Hola Mundo
### Script servidor

In [1]:
import socket

HOST = '127.0.0.1'  # Direccion loopback (localhost)
PORT = 65432        # Puerto de escucha (non-privileged ports > 1023)

# AF_INET: Address Family Intenet IP v4
# SOCK_STREAM: TCP Protocol

with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
    # Asocia un Host y Puerto a un Socket
    s.bind((HOST, PORT))
    # Coloca el socket en modo escucha
    s.listen()
    # Acepta conexiones entrantes
    conn, addr = s.accept()    # <- Blocking function (retorna socket, direccion)

    with conn:
        print('Conectado a', addr)
        while True:
            # Recibe datos en un buffer de 1024 bytes
            data = conn.recv(1024)
            if not data:
                break
            
            # Si hay datos en el buffer, broadcast...
            conn.sendall(data)
    

### Script cliente

In [ ]:
import socket

HOST = '127.0.0.1'  # Server hostname o Direccion IP
PORT = 65432        # El puerto usado por el servidor

with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
    # Utilizar el socket para conectarse a una direccion y puerto
    s.connect((HOST, PORT))
    # Enciar data binaria
    s.sendall(b'Hola mundo')
    # Recibir datos en un buffer de 1024
    data = s.recv(1024)

print('Recibido', repr(data))

## Intrercambio de información (Cliente - Sercvidor)
Las comunicaciones en red se sirven de un modelo llamado Cliente-Servidor, donde los Clientes son los elementos que se 
comunicarán entre si, mientras que el Servidor será el intermediario en el intercambio de información entre los Clientes. Para esto sera necesario que el Servidor pueda recibir la información del Cliente y este enviarla de forma inmediata.

### Script Servidor

In [ ]:
import socket

HOST = '127.0.0.1'  # Direccion loopback (localhost)
PORT = 65432        # Puerto de escucha (non-privileged ports > 1023)

with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
    s.bind((HOST, PORT))
    s.listen()
    conn, addr = s.accept()
    
    try:
        with conn:
            print("Conectado a", addr)
            while True:
                data = conn.recv(1)
                
                if not data:
                    break
                else:
                    strData = data.decode('utf-8')
                    print(strData)
                          
    except KeyboardInterrupt:
        pass
    
print("Cerrando conexion")

### Script Cliente

In [ ]:
import socket
import time

HOST = '127.0.0.1'  # Server hostname o Direccion IP
PORT = 65432        # El puerto usado por el servidor

with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
    s.connect((HOST, PORT))
    
    for i in range(1, 11):
        strData = str(i)
        data = strData.encode('utf-8')
        s.send(data)
        
        time.sleep(1)

## Header de comunicación: Agregando encabezado a la transmisión para la señalización
En el ejemplo anterior, se puede ver que el servidor recibe los datos de forma inmediata 1 byte a la vez, lo que hace que se tomen los bytes de manera rapida pero será necesario recomponer el mensaje original. Para obtener velocidad y control del mensaje, lo ideal es que el buffer del Server será variable y su tamaño este en función de mensaje a ser enviado por el Cliente

Definiremos un protocolo de comunicación que incluirá un Header de 10 catacteres (bytes), en donde se especificará en número de caracteres (bytes) que serán enviados al Server.

### Script Servidor

In [ ]:
import socket

HOST = '127.0.0.1'  # Direccion loopback (localhost)
PORT = 65432        # Puerto de escucha (non-privileged ports > 1023)
HEADER = 10

with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
    s.bind((HOST, PORT))
    s.listen()
    conn, addr = s.accept()
    
    try:
        with conn:
            print("Conectado a", addr)
            while True:
                data_len = conn.recv(HEADER)
                
                if not data_len:
                    break
                else:
                    data = conn.recv(int(data_len))
                    strData = data.decode('utf-8')
                    print(strData)
                          
    except KeyboardInterrupt:
        pass
    
print("\nCerrando conexion")

### Script Cliente

In [ ]:
import socket
import time

HOST = '127.0.0.1'  # Server hostname o Direccion IP
PORT = 65432        # El puerto usado por el servidor
HEADER = 10

with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
    s.connect((HOST, PORT))
    
    names = ['Maria',
             'Rodrigo',
             '',
             'Juan',
             'Nabucodonosor',
             'Sandro']
    
    for i in names:
        # FORMATO: <HEADER><DATA>
        strData = str(i)        
        data_len = str(len(strData))
        
        data = f"{data_len:<{HEADER}}{strData}".encode('utf-8')
        
        s.send(data)
        
        time.sleep(1)

## Serialización: el módulo `pickle`
El módulo `pickle` permite "serializar" un dato (mashalling), esto es convertír un objeto (como puede ser una lista, un diccionario, un arreglo, un archivo, etc) en una trama que tal forma que pueda enviarse por un canal de comunicación.

### Script Servidor

In [ ]:
import socket
import pickle

HOST = '127.0.0.1'  # Direccion loopback (localhost)
PORT = 65432        # Puerto de escucha (non-privileged ports > 1023)
HEADER = 10

with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
    s.bind((HOST, PORT))
    s.listen()
    conn, addr = s.accept()
    +
    try:
        with conn:
            print("Conectado a", addr)
            while True:
                data_len = conn.recv(HEADER)
              
                if not data_len:
                    break
                else:
                    data = b''
                    data += conn.recv(int(data_len))
                    data_deserial = pickle.loads(data)
                    print(data_deserial)
                          
    except KeyboardInterrupt:
        pass
    
print("\nCerrando conexion")

### Script Cliente

In [ ]:
import socket
import pickle

HOST = '127.0.0.1'  # Server hostname o Direccion IP
PORT = 65432        # El puerto usado por el servidor
HEADER = 10

with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
    s.connect((HOST, PORT))
    
    names = [{'nombre': 'Elvio', 'apellido': 'Lado'}, 
             ('ene', 'feb', 'mar'), 
             1+3j]
    
    data_serial = pickle.dumps(names)   # dumps: vuelca a una trama binaria
    
    # FORMATO: <HEADER><DATA>
    data_len = str(len(data_serial))
        
    data = bytes(f"{data_len:<{HEADER}}", 'utf-8') + data_serial
    print(data)
    s.send(data)

## Aplicación: Chat Service (Client/Server)

In [ ]:
import socket
import threading
import sys
 
HOST = '127.0.0.1'
PORT = 5000
HEADER_SIZE = 10

class Server:     
    def __init__(self):
        # Lista con los sockets de los clientes
        self.connections = []
        # Se establece el socket del servidor (socket, bind, listen)
        self.sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        # Permite eliminar el error "socket already in use"
        self.sock.setsockopt(socket.SOL_SOCKET, socket.SO_REUSEADDR, 1)
        self.sock.bind((HOST, PORT))
        self.sock.listen()
   
    
    def run(self):
        print("Servidor iniciado. Esperando conexiones...")
        while True:
            # Se aceptan las conexiones entrantes
            conn, addr = self.sock.accept()
            
            # Se levanta el thread de manejo de las conexiones entrantes
            th = threading.Thread(target=self.handler, args=(conn, addr), daemon=True)
            th.start()
            
            # Informa sobre la conexion entrante
            print(str(addr[0]) + ':' + str(addr[1]), "connected")
            
            # Se agrega el socket cliente a la lista de conexiones
            self.connections.append(conn)
   
         
    def handler(self, conn, addr):
        while True:
            # Si es que no hay problemas con la conexion del cliente...
            try:
                # Lee el encabezdo para el buffer y los datos entrantes
                data_header = conn.recv(HEADER_SIZE)
                data = conn.recv(int(data_header))
                
                # Hace ub broadcast del dato entrante a los otros sockets
                for connection in self.connections:
                    connection.send(data_header + data)
                    
            # Si hay problemas con la conexion del cliente...
            except:
                # El cliente se ha desconectado. Informar y eliminar a conexionj
                print(str(addr[0]) + ':' + str(addr[1]), "disconencted")
                self.connections.remove(conn)
                conn.close()
                break
                
        
class Client:
    def __init__(self, address, username="chat_user"):
        # Nombre del usuario del chat
        self.username = username
        # Se establece el socket del cliente (socket, connect)
        self.sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        self.sock.connect((address, PORT))
        
        # Se levanta el thread que gestiona el envio de mensajes
        th = threading.Thread(target=self.sendMsg, daemon=True)
        th.start()
        
        # Lazo de recepcion de mensajes
        while True:
            # Recibe el encabezado de los mensajes entrantes
            data_header = self.sock.recv(HEADER_SIZE)
            
            # Si no hay datos entrantes se cancela el proceso
            if not data_header:
                break
            
            # De lo contrario, se recibe el mensaje y se muestra
            # "username > mensaje"
            data = self.sock.recv(int(data_header))
            print(data.decode('utf-8'))
             
     
    def sendMsg(self):
        while True:
            # Se pide al usuario que ingrese el mensaje y se encapsula con un header
            # para su envio
            strData = input(f"{self.username}> ")
            data_len = len(strData + self.username + "> ")
                
            self.sock.send(f"{data_len:<{HEADER_SIZE}}{self.username}> {strData}".encode('utf-8'))
             

def main():             
    if (len(sys.argv) > 1):
        if len(sys.argv) == 3:
            client = Client(sys.argv[1], sys.argv[2])
        else:
            client = Client(sys.argv[1])
    else:
        server = Server()
        server.run()
        
        
if __name__ == "__main__":
    main()

## Projecto Final - Parte 2
Escriba un programa en tkinter de Chat TCP/IP. Este debe de permitir establecer comunicación entre uno o varios clientes utiliando los servicios de un servidor de chat. El cliente debe de tener un GUI semajante a la aplicación Chat Serial, pero en lugar de seleccionar un puerto serie de conexión, deberá ingresar la dirección IP del servidor y el número de puertos en dos Entry para establecer la conexión.

Establezca los controles y mensajes de error, status y diseño de la aplicación. Este es un proyecto abierto por lo que cada detalle que agregue y sea construictivo afecta la calidad del producto y por lo tanto la valoración del mismo.